In [1]:

import keras
import keras.applications
from keras import Model, layers
from keras.layers import Input,Conv2D

Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50

In [3]:
from keras.optimizers import Adam
tensor_input = Input(shape=(128, 128, 3))
model = ResNet50(input_tensor = tensor_input ,include_top=False,weights=None)
x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(6, activation='softmax')(x)
model = Model(model.input, predictions)


Instructions for updating:
Colocations handled automatically by placer.


/home/anmesh/.conda/envs/crystal/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from dataLoader import data_loader

filepath = "weights-impVGG-{epoch:02d}-{val_acc:.2f}.hdf5"
#load any previous weight files \
# Model.load_weights("weights-imp-18-0.73.hdf5")

dataLoader = data_loader('data/train', 'data/val', 'data/test')
train_generator, validation_generator, test_generator = dataLoader.load_images()
checkpoint_1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint_2 =  TensorBoard(log_dir='logs/{}', histogram_freq=0, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0)
callbacks_list = [checkpoint_2,checkpoint_1,reduce_lr]
#NO. of Steps per epoch 
train_steps = np.math.ceil(train_generator.samples / train_generator.batch_size)
val_steps = np.math.ceil(validation_generator.samples / validation_generator.batch_size)
test_steps = np.math.ceil(test_generator.samples / test_generator.batch_size)

history = model.fit_generator(train_generator, steps_per_epoch =train_steps, epochs = 20,callbacks=callbacks_list,validation_data=test_generator,validation_steps = val_steps)
print('--------Test data--------')

x = model.evaluate_generator(test_generator, steps = test_steps,verbose = 1)
print(x)

Found 42000 images belonging to 6 classes.
Found 9000 images belonging to 6 classes.
Found 9000 images belonging to 6 classes.


In [ ]:
model.summary()